In [1]:
import pandas as pd
import numpy as np

#read data from csv
data = pd.read_csv("data/training_data_utf8.csv")

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#define function to transform long/lat to distance
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

In [3]:
#define long/lat center of vienna
latitude_center = 48.210033
longitude_center = 16.363449

In [4]:
#create new column distance_to_center
for index, row in data.iterrows():
    data.loc[index, 'distance_to_center'] = haversine(row['longitude'], row['latitude'], longitude_center, latitude_center)

In [5]:
#extract year from transaction_ym
date = data.transaction_ym.apply(pd.to_datetime, format="%Y-%m")

data["year"] = date.dt.year

In [6]:
#create new column relative_share
relative_share = data.plot_area * data.plot_share
data["relative_share"] = relative_share

In [7]:
#fill NA/NaN with mean
data = data.fillna(data.mean())

In [8]:
data["land_use"] = data["land_use"].astype(str)
data["district"] = data["district"].astype(str)
data["property_type"] = data["property_type"].astype(str)
data["contract_type"] = data["contract_type"].astype(str)
data["protection_zone"] = data["protection_zone"].astype(str)
data["building_ban"] = data["building_ban"].astype(str)

In [9]:
#split data into training and test data
data_train = data[data.year <= 2010]
data_test = data[data.year > 2010]

In [10]:
#filter outliers
data_train = data_train[data.distance_to_center < 15]
#data_train = data_train[data.price_per_m2 < 10000]
data.shape

/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(44366, 21)

In [11]:
#display shape of training data
data_train.shape

(40047, 21)

In [12]:
#display shape of testing data
data_test.shape

(4316, 21)

In [13]:
#define numerical and categorical pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from future_encoders import OneHotEncoder

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('one_hot', OneHotEncoder())
])

In [14]:
#define features for pipeline A
#pip_data_a = data[["building_floors", "year", "distance_to_center", "land_use", "relative_share", "district"]]
pip_data_a = data[["building_floors", "year", "distance_to_center", "relative_share"]]
pip_data_a_num = data[["year", "distance_to_center", "building_floors", "relative_share"]]
#pip_data_a_cat = data[["land_use", "district"]]

In [15]:
#define full_pipeline_a for training data_a
from future_encoders import ColumnTransformer

num_attribs = list(pip_data_a_num)
#cat_attribs = list(pip_data_a_cat)

full_pipeline_a = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
#    ('cat', cat_pipeline, cat_attribs)
])

full_pipeline_a.fit_transform(pip_data_a)

array([[ 1.72687487,  1.07481832, -0.86724244,  0.2488543 ],
       [ 1.72687487,  1.61541514, -0.05170695, -0.0859057 ],
       [ 1.72687487,  0.89049187, -0.05170695, -0.0879487 ],
       ...,
       [-1.55580778, -0.94233947,  0.        , -0.01449802],
       [-1.55580778,  0.        ,  0.        ,  0.08502523],
       [-1.55580778,  0.        ,  0.        ,  0.71640926]])

In [16]:
#define training data for pipeline A
data_train_a = data_train[["price_per_m2", "building_floors", "year", "distance_to_center", "land_use", "relative_share", "district"]]

data_train_labels_a = data_train_a["price_per_m2"].copy()
data_train_a = data_train_a.drop("price_per_m2", axis=1)

In [17]:
#define features for pipeline B
pip_data_b = data[["price_per_m2", "property_type","district","contract_type","protection_zone","building_ban", "building_floors", "building_year", "land_use"]]
pip_data_b_num = data[["building_floors", "building_year"]]
pip_data_b_cat = data[["property_type","district","contract_type","protection_zone","building_ban", "land_use"]]

In [18]:
#define full_pipeline_b for training data_b
num_attribs = list(pip_data_b_num)
cat_attribs = list(pip_data_b_cat)

full_pipeline_b = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs)
])

full_pipeline_b.fit_transform(pip_data_b)

<44366x114 sparse matrix of type '<class 'numpy.float64'>'
	with 328984 stored elements in Compressed Sparse Row format>

In [19]:
#define training data for pipeline B
data_train_b = data_train[["price_per_m2", "property_type","district","contract_type","protection_zone","building_ban", "building_floors", "building_year", "land_use"]]
data_train_b_num = data_train[["building_floors", "building_year"]]
data_train_b_cat = data_train[["property_type","district","contract_type","protection_zone","building_ban", "land_use"]]

data_train_labels_b = data_train_b["price_per_m2"].copy()
data_train_b = data_train_b.drop("price_per_m2", axis=1)

In [20]:
#fit data_a and data_b
data_train_prep_a = full_pipeline_a.transform(data_train_a)
data_train_prep_b = full_pipeline_b.transform(data_train_b)

In [21]:
#declare diplay_scores function for model evaluation
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation", scores.std())

In [22]:
#train and cv linear regression for data_prep_a
from sklearn.linear_model import LinearRegression

lin_reg_a = LinearRegression()
lin_reg_a.fit(data_train_prep_a, data_train_labels_a)

scores = cross_val_score(lin_reg_a, data_train_prep_a, data_train_labels_a, scoring="neg_mean_squared_error", cv=10)
lin_reg_rmse_scores_a = np.sqrt(-scores)

display_scores(lin_reg_rmse_scores_a)

Scores: [1938.78364946 2727.41746368 1626.53015499 3963.99566826 1193.14384089
 2137.41860248 2439.89859553 1171.36736828  828.87043603  687.20217617]
Mean: 1871.4627955760286
Standard Deviation 949.1812428222001


In [23]:
#train and cv linear regression for data_prep_b
lin_reg_b = LinearRegression()
lin_reg_b.fit(data_train_prep_b, data_train_labels_b)

scores = cross_val_score(lin_reg_b, data_train_prep_b, data_train_labels_b, scoring="neg_mean_squared_error", cv=10)
lin_reg_rmse_scores_b = np.sqrt(-scores)

display_scores(lin_reg_rmse_scores_b)

Scores: [1530.62318724 2333.04066478 1292.97300724 3838.08286767  965.79040051
 2037.78050413 2345.13054515  881.67957723  755.61050253  877.97949989]
Mean: 1685.8690756368073
Standard Deviation 920.3853456417961


In [24]:
#train and cv ridge regression for data_prep_a
from sklearn.linear_model import Ridge

ridge_reg_a = Ridge(alpha=0.9)
ridge_reg_a.fit(data_train_prep_a, data_train_labels_a)

scores = cross_val_score(ridge_reg_a, data_train_prep_a, data_train_labels_a, scoring="neg_mean_squared_error", cv=10)
ridge_reg_rmse_scores_a = np.sqrt(-scores)

display_scores(ridge_reg_rmse_scores_a)

Scores: [1938.78711437 2727.42103499 1626.5286074  3963.99477306 1193.12659863
 2137.41564064 2439.89866815 1171.3669052   828.87023997  687.20317992]
Mean: 1871.4612762335692
Standard Deviation 949.1825160601202


In [25]:
#train and cv ridge regression for data_prep_b
ridge_reg_b = Ridge(alpha=0.9)
ridge_reg_b.fit(data_train_prep_b, data_train_labels_b)

scores = cross_val_score(ridge_reg_b, data_train_prep_b, data_train_labels_b, scoring="neg_mean_squared_error", cv=10)
ridge_reg_rmse_scores_b = np.sqrt(-scores)

display_scores(ridge_reg_rmse_scores_b)

Scores: [1532.09097864 2334.20291848 1292.84118436 3837.92273394  965.08461571
 2037.52001112 2344.93578261  881.13481921  754.73577746  876.84057158]
Mean: 1685.7309393097203
Standard Deviation 920.6780795024581


In [26]:
#train and cv lasso regression for data_prep_a
from sklearn.linear_model import Lasso

lasso_reg_a = Lasso(alpha=0.5)
lasso_reg_a.fit(data_train_prep_a, data_train_labels_a)

scores = cross_val_score(lasso_reg_a, data_train_prep_a, data_train_labels_a, scoring="neg_mean_squared_error", cv=10)
lasso_reg_rmse_scores_a = np.sqrt(-scores)

display_scores(lasso_reg_rmse_scores_a)

Scores: [1938.91993217 2727.57792782 1626.41862789 3963.98621934 1192.64587886
 2137.3447334  2439.90564031 1171.22639213  828.91361439  687.42564777]
Mean: 1871.4364614074243
Standard Deviation 949.2092057817932


In [27]:
#train and cv lasso regression for data_prep_b
lasso_reg_b = Lasso(alpha=0.5)
lasso_reg_b.fit(data_train_prep_b, data_train_labels_b)

scores = cross_val_score(lasso_reg_b, data_train_prep_b, data_train_labels_b, scoring="neg_mean_squared_error", cv=10)
lasso_reg_rmse_scores_b = np.sqrt(-scores)

display_scores(lasso_reg_rmse_scores_b)

Scores: [1541.6171925  2340.68880413 1291.44023273 3837.99417163  962.94445036
 2037.64196021 2343.41541825  879.52598942  749.62074051  871.59892769]
Mean: 1685.6487887443704
Standard Deviation 922.2435089845777


In [28]:
#train and cv decision tree regression for data_prep_a
from sklearn.tree import DecisionTreeRegressor

tree_reg_a = DecisionTreeRegressor()
tree_reg_a.fit(data_train_prep_a, data_train_labels_a)

#scores = cross_val_score(tree_reg_a, data_train_prep_a, data_train_labels_a, scoring="neg_mean_squared_error", cv=10)
#tree_reg_rmse_scores_a = np.sqrt(-scores)

#display_scores(tree_reg_rmse_scores_a)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [29]:
#train and cv decision tree regression for data_prep_b
tree_reg_b = DecisionTreeRegressor()
tree_reg_b.fit(data_train_prep_b, data_train_labels_b)

#scores = cross_val_score(tree_reg_b, data_train_prep_b, data_train_labels_b, scoring="neg_mean_squared_error", cv=10)
#tree_reg_rmse_scores_b = np.sqrt(-scores)

#display_scores(tree_reg_rmse_scores_b)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [30]:
#train and cv random forest regression for data_prep_a
from sklearn.ensemble import RandomForestRegressor

forest_reg_a = RandomForestRegressor()
forest_reg_a.fit(data_train_prep_a, data_train_labels_a)

#scores = cross_val_score(forest_reg_a, data_train_prep_a, data_train_labels_a, scoring="neg_mean_squared_error", cv=10)
#forest_reg_rmse_scores_a = np.sqrt(-scores)

#display_scores(forest_reg_rmse_scores_a)

/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [31]:
#train and cv random forest regression for data_prep_b
forest_reg_b = RandomForestRegressor()
forest_reg_b.fit(data_train_prep_b, data_train_labels_b)

#scores = cross_val_score(forest_reg_b, data_train_prep_b, data_train_labels_b, scoring="neg_mean_squared_error", cv=10)
#forest_reg_rmse_scores_b = np.sqrt(-scores)

#display_scores(forest_reg_rmse_scores_b)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [32]:
#prepare test_data_a for testing
#test_data_a = data_test[["price_per_m2", "building_floors", "year", "distance_to_center", "land_use", "relative_share", "district"]]
test_data_a = data_test[["price_per_m2", "building_floors", "year", "distance_to_center", "relative_share"]]

test_labels_a = test_data_a["price_per_m2"].copy()
test_set_a = test_data_a.drop("price_per_m2", axis=1)

test_data_prep_a = full_pipeline_a.transform(test_set_a)

#prepare test_data_b for testing
test_data_b = data_test[["price_per_m2", "property_type","district","contract_type","protection_zone","building_ban", "building_floors", "building_year", "land_use"]]

test_labels_b = test_data_b["price_per_m2"].copy()
test_set_b = test_data_b.drop("price_per_m2", axis=1)

test_data_prep_b = full_pipeline_b.transform(test_set_b)

In [33]:
#prepare predictions on test set

#predictions from A-Models
lin_reg_pred_a = lin_reg_a.predict(test_data_prep_a)
ridge_reg_pred_a = ridge_reg_a.predict(test_data_prep_a)
lasso_reg_pred_a = lasso_reg_a.predict(test_data_prep_a)
tree_reg_pred_a = tree_reg_a.predict(test_data_prep_a)
forest_reg_pred_a = forest_reg_a.predict(test_data_prep_a)

#predictions from B-Models
lin_reg_pred_b = lin_reg_b.predict(test_data_prep_b)
ridge_reg_pred_b = ridge_reg_b.predict(test_data_prep_b)
lasso_reg_pred_b = lasso_reg_b.predict(test_data_prep_b)
tree_reg_pred_b = tree_reg_b.predict(test_data_prep_b)
forest_reg_pred_b = forest_reg_b.predict(test_data_prep_b)

In [34]:
#used metrics for experiment
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [35]:
#get metrics for models

#metrics for A-Models
lin_mse_a = np.sqrt(mean_squared_error(test_labels_a, lin_reg_pred_a))
lin_mae_a = mean_absolute_error(test_labels_a, lin_reg_pred_a)
ridge_mse_a = np.sqrt(mean_squared_error(test_labels_a, ridge_reg_pred_a))
ridge_mae_a = mean_absolute_error(test_labels_a, ridge_reg_pred_a)
lasso_mse_a = np.sqrt(mean_squared_error(test_labels_a, lasso_reg_pred_a))
lasso_mae_a = mean_absolute_error(test_labels_a, lasso_reg_pred_a)
tree_mse_a = np.sqrt(mean_squared_error(test_labels_a, tree_reg_pred_a))
tree_mae_a = mean_absolute_error(test_labels_a, tree_reg_pred_a)
forest_mse_a = np.sqrt(mean_squared_error(test_labels_a, forest_reg_pred_a))
forest_mae_a = mean_absolute_error(test_labels_a, forest_reg_pred_a)


#metrics for B-Models
lin_mse_b = np.sqrt(mean_squared_error(test_labels_b, lin_reg_pred_b))
lin_mae_b = mean_absolute_error(test_labels_b, lin_reg_pred_b)
ridge_mse_b = np.sqrt(mean_squared_error(test_labels_b, ridge_reg_pred_b))
ridge_mae_b = mean_absolute_error(test_labels_b, ridge_reg_pred_b)
lasso_mse_b = np.sqrt(mean_squared_error(test_labels_b, lasso_reg_pred_b))
lasso_mae_b = mean_absolute_error(test_labels_b, lasso_reg_pred_b)
tree_mse_b = np.sqrt(mean_squared_error(test_labels_b, tree_reg_pred_b))
tree_mae_b = mean_absolute_error(test_labels_b, tree_reg_pred_b)
forest_mse_b = np.sqrt(mean_squared_error(test_labels_b, forest_reg_pred_b))
forest_mae_b = mean_absolute_error(test_labels_b, forest_reg_pred_b)

In [36]:
df_a = pd.DataFrame(columns=['Linear/RMSE', 'Linear/MAE', 'Ridge/RMSE', 'Ridge/MAE', 'Lasso/RMSE', 'Lasso/MAE', 'Tree/RMSE', 'Tree/MAE', 'Forest/RMSE', 'Forest/MAE'])
df_a['Linear/RMSE'] = [lin_mse_a]
df_a['Linear/MAE'] = [lin_mae_a]
df_a['Ridge/RMSE'] = [ridge_mse_a]
df_a['Ridge/MAE'] = [ridge_mae_a]
df_a['Lasso/RMSE'] = [lasso_mse_a]
df_a['Lasso/MAE'] = [lasso_mae_a]
df_a['Tree/RMSE'] = [tree_mse_a]
df_a['Tree/MAE'] = [tree_mae_a]
df_a['Forest/RMSE'] = [forest_mse_a]
df_a['Forest/MAE'] = [forest_mae_a]

df_a

,Linear/RMSE,Linear/MAE,Ridge/RMSE,Ridge/MAE,Lasso/RMSE,Lasso/MAE,Tree/RMSE,Tree/MAE,Forest/RMSE,Forest/MAE
0,2318.899308,941.967807,2318.90442,941.964625,2319.154941,941.640439,2307.806231,899.126776,1972.363086,751.786161


In [37]:
df_b = pd.DataFrame(columns=['Linear/RMSE', 'Linear/MAE', 'Ridge/RMSE', 'Ridge/MAE', 'Lasso/RMSE', 'Lasso/MAE', 'Tree/RMSE', 'Tree/MAE', 'Forest/RMSE', 'Forest/MAE'])
df_b['Linear/RMSE'] = [lin_mse_b]
df_b['Linear/MAE'] = [lin_mae_b]
df_b['Ridge/RMSE'] = [ridge_mse_b]
df_b['Ridge/MAE'] = [ridge_mae_b]
df_b['Lasso/RMSE'] = [lasso_mse_b]
df_b['Lasso/MAE'] = [lasso_mae_b]
df_b['Tree/RMSE'] = [tree_mse_b]
df_b['Tree/MAE'] = [tree_mae_b]
df_b['Forest/RMSE'] = [forest_mse_b]
df_b['Forest/MAE'] = [forest_mae_b]

df_b

,Linear/RMSE,Linear/MAE,Ridge/RMSE,Ridge/MAE,Lasso/RMSE,Lasso/MAE,Tree/RMSE,Tree/MAE,Forest/RMSE,Forest/MAE
0,2076.23095,789.745183,2076.675607,789.509025,2083.244149,790.764205,2157.282174,756.880797,2001.998696,706.357498


In [38]:
sorted(zip(tree_reg_b.feature_importances_, num_attribs + cat_attribs), reverse=True)

[(0.2203326818083035, 'building_floors'),
 (0.18982135849271176, 'building_year'),
 (0.018843047457558056, 'contract_type'),
 (0.006784407434691651, 'property_type'),
 (0.0066984514850143215, 'protection_zone'),
 (0.004529939587827239, 'building_ban'),
 (0.0020746424295916845, 'district'),
 (4.367921497396749e-07, 'land_use')]

In [39]:
sorted(zip(tree_reg_a.feature_importances_, num_attribs + cat_attribs), reverse=True)

[(0.28029715477599854, 'property_type'),
 (0.2763450828340236, 'district'),
 (0.27515334500964417, 'building_year'),
 (0.16820441738033376, 'building_floors')]

In [40]:
sorted(zip(forest_reg_b.feature_importances_, num_attribs + cat_attribs), reverse=True)

[(0.23210839102650596, 'building_floors'),
 (0.18374955080972669, 'building_year'),
 (0.023572314131849757, 'contract_type'),
 (0.010525779072539643, 'property_type'),
 (0.007623787163392931, 'protection_zone'),
 (0.004967878319576644, 'building_ban'),
 (0.0015513244786369393, 'district'),
 (3.7753466355842547e-07, 'land_use')]

In [41]:
sorted(zip(forest_reg_a.feature_importances_, num_attribs + cat_attribs), reverse=True)

[(0.4476955913698064, 'building_year'),
 (0.26914884057213506, 'district'),
 (0.18604443599990966, 'property_type'),
 (0.09711113205814878, 'building_floors')]

In [42]:
pd.concat([df_a, df_b])

,Linear/RMSE,Linear/MAE,Ridge/RMSE,Ridge/MAE,Lasso/RMSE,Lasso/MAE,Tree/RMSE,Tree/MAE,Forest/RMSE,Forest/MAE
0,2318.899308,941.967807,2318.904420,941.964625,2319.154941,941.640439,2307.806231,899.126776,1972.363086,751.786161
0,2076.230950,789.745183,2076.675607,789.509025,2083.244149,790.764205,2157.282174,756.880797,2001.998696,706.357498
